In [ ]:
# html에서 태그 선택하기 
#CSS selector  
#  선택자  > 자식
#  선택 자손       
#  .클래스,   #ID, 
#  선택자[속성1='속성값1'] 
# 선택되는게 한개인지, 여러개인지, 없는지 구별잘하기.

In [ ]:
#학습 목표 : Beautifulsoup 사용해서 웹 사이트의 원하는 데이터를 수집한다.
# requests : 웹 사이트로부터 데이터(html, json 등)를 얻는  lib
# Beautifulsoup는 html을 parsing해주는 lib

import requests

response=requests.get("https://www.naver.com")
print(type(response) )
print("status : ", response.status_code )
print("header : ", response.headers )
print("-------------------------------------")
print("html : ", response.text )




In [ ]:
import requests
params= {"pageNo" : 1 , "rangType" : "ALL" , "orderBy" : "sim", "keyword" : "뉴진스" }
url="https://section.blog.naver.com/Search/Post.naver"  #요거 치지마시고 공유폴더에 올려놨습니다.
response= requests.get(url, params=params)  #파라미터를 넘길수 있다.
#requests.post()  포스트방식은 url로 파라미터 전달 못함
print(type(response) )
print("status : ", response.status_code )
print("header : ", response.headers )
print("-------------------------------------")
print("html : ", response.text )


In [ ]:
# 네이버 블로그에서 얻은 html을 해석해서, 제목만 출력해보자.
import requests
from bs4 import BeautifulSoup as bs

# params= {"pageNo" : 1 , "rangType" : "ALL" , "orderBy" : "sim", "keyword" : "뉴진스" }
# url="https://section.blog.naver.com/Search/Post.naver"  #요거 치지마시고 공유폴더에 올려놨습니다.
# response= requests.get(url, params=params)  #파라미터를 넘길수 있다.
# html= bs(response.text, 'html.parser')  # Beautifulsoup에선 html뿐만아니라 xml 등도 parsing 해줍니다. xhtml.parser..
# # html.select_one()
# # html.select()   # CSS selector를 통해원하는 태그를 얻을 수 있습니다.
# div=html.select_one( "content")
# print(div)


In [ ]:
# 네이버 삼성전자 검색 - 뉴스 탭    제목만 출력해보기.
import requests
from bs4 import BeautifulSoup as bs

url="https://search.naver.com/search.naver?ssc=tab.news.all&where=news&sm=tab_jum&query=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90"
response=requests.get(url)
html= bs(response.text, 'html.parser')
li_list= html.select(".list_news .bx" )    #li태그안에 li태그가 또 있었네..
for li in li_list :
    aTag=li.select_one("a[title]")  # title속성 이 있는 a태그
    print(aTag.text)
    print()


In [ ]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

url="https://en.wikipedia.org/wiki/List_of_most-visited_websites"  #위키피디아
response=requests.get(url)
html= bs(response.text, 'html.parser') 
table=html.select_one("table")
tbody= table.select_one("tbody")
tr_list=tbody.select("tr")

rank_df=pd.DataFrame(columns=("Website","DomainName","ranking","Type" , "Company","Country") )

for tr in tr_list[2:] : 
    website=tr.select_one("th").text
    td_list= tr.select("td")
    domain_name=td_list[0].text
    ranking= td_list[1].text   #1,2,3,4,5,6
    type= td_list[3].text    
    company= td_list[4].text
    country= td_list[5].text
    rank_df.loc[ranking]=  [ website, domain_name,ranking, type,company ,country]
print(rank_df)

In [14]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

url="https://music.bugs.co.kr/chart?wl_ref=M_left_02_01"
response=requests.get(url)
html= bs(response.text, 'html.parser') 
table=html.select_one(".byChart")
tr_list=table.select("tr")
rank_df=pd.DataFrame(columns=("rank","title","artist"))
for tr in tr_list[1:] : 
    rank=tr.select_one("strong").text
    title=tr.select("a[title]")[0].text
    artist=tr.select("a[title]")[1].text
    rank_df.loc[rank] =[rank,title,artist]

print(rank_df)


    rank                  title              artist
1      1               Magnetic          아일릿(ILLIT)
2      2        나는 아픈 건 딱 질색이니까             (여자)아이들
3      3       첫 만남은 계획대로 되지 않아           TWS (투어스)
4      4                    밤양갱            비비(BIBI)
5      5    Welcome to the Show         DAY6 (데이식스)
..   ...                    ...                 ...
96    96  이브, 프시케 그리고 푸른 수염의 아내  LE SSERAFIM (르세라핌)
97    97                     일기                 김나영
98    98                   심(心)             DK(디셈버)
99    99                   STAY       The Kid LAROI
100  100             너의 편이 돼 줄게                 다비치

[100 rows x 3 columns]


In [32]:
#네이버 증권도 됨
#네이버 시리즈온은 됨
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
url="https://finance.naver.com/sise/"
response=requests.get(url)
html= bs(response.text, 'html.parser') 
ul=html.select_one("#popularItemList")
li_list=ul.select("li")
df=pd.DataFrame(columns=("전일","시가","고가","저가","거래량","거래대금"))


for li in li_list :
    a=li.select_one("a")
    print(a.text)
    suburl="https://finance.naver.com" + a.attrs["href"]
    print(suburl)
    sub_response=requests.get(suburl)
    sub_html=bs(sub_response.text, 'html.parser')
    #똑같이 여러분들이  select, select_one 써서 원하는 데이터 얻으시면 됩니다
    #전일,시가, 고가,저가 , 거래량, 거래대금  을 pd로 넣으면 될거고.  인덱스는 이름으로 a.text
    table= sub_html.select_one("table[summary]")
    tbody=table.select_one("tbody")
    tr_list= table.select("tr")

    #위 tr 전일 고가 거래량
    td_list0=tr_list[0].select("td") 
    yesterday=td_list0[0].select_one(".blind").text  #전일
    high= td_list0[1].select_one(".blind").text  #고가
    trade_amount= td_list0[2].select_one(".blind").text  #거래량


    #아래 tr 시가 저가 거래대금
    td_list1=tr_list[1].select("td") 
    price=td_list1[0].select_one(".blind").text  #시가
    low= td_list1[1].select_one(".blind").text  #저가
    trade_amount_price= td_list1[2].select_one(".blind").text+"백만"  #거래대금


    df.loc[a.text]= [yesterday,high,trade_amount , price,low,trade_amount_price]
    

df


삼성전자
https://finance.naver.com/item/main.naver?code=005930
HLB
https://finance.naver.com/item/main.naver?code=028300
SK하이닉스
https://finance.naver.com/item/main.naver?code=000660
엔켐
https://finance.naver.com/item/main.naver?code=348370
NAVER
https://finance.naver.com/item/main.naver?code=035420
에프에스티
https://finance.naver.com/item/main.naver?code=036810
기아
https://finance.naver.com/item/main.naver?code=000270
흥구석유
https://finance.naver.com/item/main.naver?code=024060
현대차
https://finance.naver.com/item/main.naver?code=005380
에코프로비엠
https://finance.naver.com/item/main.naver?code=247540


,전일,시가,고가,저가,거래량,거래대금
삼성전자,"83,700","83,200","26,110,522","82,900","81,200","2,142,373백만"
HLB,"93,000","99,900","2,468,980","92,000","90,300","237,423백만"
SK하이닉스,"187,400","188,400","2,635,889","185,400","183,000","489,741백만"
엔켐,"361,500","350,000","980,849","347,000","316,000","325,937백만"
NAVER,"185,900","184,600","622,372","183,800","182,600","114,005백만"
에프에스티,"28,250","35,700","5,616,164","27,500","27,450","181,792백만"
기아,"109,800","115,000","2,433,618","109,200","107,200","273,861백만"
흥구석유,"15,550","19,110","19,218,069","18,200","16,390","346,238백만"
현대차,"238,000","243,000","1,007,483","236,000","233,500","241,192백만"
에코프로비엠,"228,500","233,500","529,167","222,500","222,000","120,886백만"


In [3]:
# beautifulsoup  마지막
# for index,item in  enumerate(반복가능한거 ) : 
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
url="https://serieson.naver.com/v3/movie/ranking/realtime"
response=requests.get(url)
html= bs(response.text, 'html.parser') 





<!DOCTYPE html>
<html><head><meta charset="utf-8"/><meta content="width=device-width" name="viewport"/><meta content="최신 영화와 방송, 네이버 시리즈온에서 감상하세요." property="og:description"/><meta content="website" property="og:type"/><meta content="https://static-serieson.pstatic.net/ssr-desktop/23.48.1/_next/static/media/og_image.32ea4a59.png" property="og:image"/><meta content="1200" property="og:image:width"/><meta content="630" property="og:image:height"/><link href="https://static-serieson.pstatic.net/ssr-desktop/23.48.1/_next/static/media/favicon_96.f49ffb78.png" rel="shortcut icon" type="image/x-icon"/><link href="https://static-serieson.pstatic.net/ssr-desktop/23.48.1/_next/static/media/favicon_48.5f5d9ea9.ico" rel="icon" type="image/x-icon"/><link href="https://static-serieson.pstatic.net/ssr-desktop/23.48.1/_next/static/media/favicon_114.45097906.png" rel="apple-touch-icon-precomposed"/><link href="https://static-serieson.pstatic.net/ssr-desktop/23.48.1/_next/static/media/favicon_114.450979